In [ ]:
!pip install -U spacy==3.1

In [ ]:
pip install wandb

In [ ]:
pip install contractions

In [1]:
import spacy
import pandas as pd
import numpy as np
import spacy.cli
import seaborn as sns
spacy.cli.download("en_core_web_md")
nlp = spacy.load("en_core_web_md")

import nltk
import string
import fasttext
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [2]:
#First dataset contains data about reviews - text

drew = pd.read_csv("reviews.csv", 
                 index_col=False,
                 low_memory=False)
drew

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27
4,GgWFjRHhelaiUgR2-H6N4g,pRPT3vqhqpU7kHgmKJamvw,-_GnwXmzC3DXsHR9nyaC2g,3,0,0,0,3.5 stars! I got the avocado margarita and it ...,2012-11-06 07:09:57
...,...,...,...,...,...,...,...,...,...
214869,Mc6gBUoqj8VNyGHBOTIPCw,_XXa8CyI4uKOdu6RRbWE4g,cs6HfZNykLVitm09jWFqWg,4,5,3,4,I went here for brunch with 2 friends on a Sun...,2018-04-17 00:28:02
214870,jDMk2EEGG-fkuA2rnCr0zg,KTgetFhipsOG1vWfVuSaiA,cs6HfZNykLVitm09jWFqWg,5,0,0,0,Our group shared dinner entrees and side dishe...,2017-10-13 21:02:12
214871,MbmrN5q_JKkokjOdqLXxhQ,SF2Hu55kX9QJDvZETuZ9eg,uXOVFQraz1va1TrtgiqYTg,4,0,0,0,This was a fun place to dine. It is a one chef...,2016-10-31 20:47:16
214872,2n2PbNLYlQSRs88VrZtrGg,_BwuIa1-qjIL1PdYB_awyA,Gzo1G_mMc2kpduvK2yQFYg,2,0,0,0,The food was ok. I order the pork bites and it...,2015-07-21 13:21:59


In [3]:
#Second dataset contains data about users who write the reviews

dus = pd.read_csv("users.csv", 
                 index_col=False,
                 low_memory=False)
dus

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,163,190,361,147,1212,5691,2541,2541,815,323
1,dIIKEfOgo0KqUfGQvGikPg,Gabi,2136,2007-08-10 19:01:51,21272,10289,18046,"2007,2008,2009,2010,2011,2012,2013,2014,2015,2...","XPzYf9_mwG2eXYP2BAGSTA, 2LooM5dcIk2o01nftYdPIg...",1025,...,87,94,232,96,1187,3293,2205,2205,472,294
2,JnPIjvC0cmooNDfsa9BmXg,Kat,987,2009-02-09 16:14:29,7234,4722,4035,"2009,2010,2011,2012,2013,2014","HQZPQhKMwRAyS6BCselVWQ, kP2U1s_sjQfHO9grxiyDTA...",420,...,129,93,219,90,1120,4510,1566,1566,391,326
3,37Hc8hr3cw0iHLoPzLK6Ow,Christine,495,2008-03-03 04:57:05,1577,727,1124,"2009,2010,2011","-Q88pZUcrfN0BLBDp-bkAQ, etPn4Pv1Gc4cRZjRgB_BOw...",47,...,19,32,16,15,77,131,310,310,98,44
4,n-QwITZYrXlKQRiV30MqNg,Natasha,229,2008-06-25 14:53:17,476,101,140,"2010,2011,2012,2013,2014,2015,2016","2ptwW5l68069vNtW8J-WOg, 4TVFuz8L4TsDtx_ObSin9g...",17,...,4,3,0,0,10,15,24,24,16,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154393,UsQ2rYu7MOahWbJbiI490Q,Nick,1,2017-08-17 00:02:25,0,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0
154394,sMYM2sASJh18dHSqqcLq4w,Kemannie,4,2018-12-18 16:11:21,0,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0
154395,BkgPC4uy_Jm-3fIilVsIPQ,Julia,1,2011-06-01 03:34:14,1,1,1,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0
154396,lB-12p7q5fCPFrjCxWKfYQ,Catherine,11,2013-12-29 19:01:03,3,1,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#Third dataset contains data about restaurants or businesses that are being reviewed

dbus = pd.read_csv("businesses.csv", 
                 index_col=False,
                 low_memory=False)
dbus

,hours.Wednesday,attributes.RestaurantsDelivery,attributes.Open24Hours,attributes.DogsAllowed,attributes.CoatCheck,postal_code,attributes.Smoking,hours.Thursday,attributes.DietaryRestrictions,city,...,hours.Sunday,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.AcceptsInsurance,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,NaN,NaN,NaN,NaN,NaN,97210,NaN,10:0-16:0,NaN,Portland,...,10:0-16:0,NaN,NaN,NaN,NaN,NaN,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",NaN,NaN,NaN
1,6:0-21:0,False,NaN,False,NaN,2128,NaN,6:0-21:0,NaN,Boston,...,6:0-21:0,"{'dessert': None, 'latenight': False, 'lunch':...",False,False,NaN,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,False,False
2,11:0-22:0,True,NaN,False,False,78735,u'no',16:30-22:0,NaN,Austin,...,11:0-22:0,"{'dessert': None, 'latenight': False, 'lunch':...",False,True,NaN,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,False,None
3,11:0-21:0,True,NaN,False,NaN,30308,NaN,11:0-21:0,NaN,Atlanta,...,12:0-19:0,"{'dessert': None, 'latenight': False, 'lunch':...",NaN,False,NaN,False,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,None
4,0:0-0:0,NaN,NaN,NaN,NaN,32827,NaN,0:0-0:0,NaN,Orlando,...,0:0-0:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,11:30-22:0,True,NaN,False,NaN,97202,NaN,11:30-22:0,NaN,Portland,...,11:30-22:0,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,NaN,True,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,True
352,11:0-18:0,None,NaN,NaN,NaN,97209,NaN,11:0-18:0,NaN,Portland,...,11:0-18:0,NaN,NaN,NaN,NaN,NaN,"{'BusinessParking': ""{'garage': True, 'street'...",NaN,True,NaN
353,9:0-0:0,None,NaN,False,False,2116,u'no',11:0-21:0,NaN,Boston,...,10:0-21:0,"{'dessert': False, 'latenight': False, 'lunch'...",False,True,NaN,True,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,True
354,16:0-2:0,True,NaN,NaN,True,2116,NaN,16:0-2:0,NaN,Boston,...,16:0-2:0,"{'dessert': False, 'latenight': True, 'lunch':...",False,False,NaN,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'False', '...",True,True,True


In [5]:
#Firstly we should merge reviews and business datasets on business_id 

dmez = pd.merge(drew, dbus, left_on='business_id', right_on='business_id', how='left')
dmez.head(6)

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,hours.Wednesday,...,hours.Sunday,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.AcceptsInsurance,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,12:0-20:0,...,12:0-21:0,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,NaN,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,None
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,11:0-2:0,...,11:0-0:0,"{'dessert': None, 'latenight': True, 'lunch': ...",False,False,NaN,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,True
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,10:30-21:0,...,10:30-21:0,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,False,NaN,False,"{'BusinessParking': ""{'garage': False, 'street...",True,True,False
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,9:0-9:15,...,9:0-9:15,NaN,NaN,NaN,NaN,NaN,"{'DogsAllowed': 'False', 'BusinessParking': ""{...",NaN,True,NaN
4,GgWFjRHhelaiUgR2-H6N4g,pRPT3vqhqpU7kHgmKJamvw,-_GnwXmzC3DXsHR9nyaC2g,3,0,0,0,3.5 stars! I got the avocado margarita and it ...,2012-11-06 07:09:57,11:0-21:0,...,9:0-15:0,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,NaN,False,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,True
5,ldGCo22ax_ZvHV8TgE27Pg,Df1kUPqooB1FrnwvXi5HsQ,Qydynzc1ApxsSkVNEgPs9A,1,0,1,0,I love ale house. I'm here all the time. The 1...,2014-07-13 18:59:44,11:0-2:0,...,11:0-2:0,"{'dessert': None, 'latenight': True, 'lunch': ...",False,True,NaN,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,True


In [6]:
#Secondly it is neccessary to merge all datasets into one 

dfin = pd.merge(dmez, dus, left_on='user_id', right_on='user_id', how='left')
dfin.head(6)


,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,hours.Wednesday,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,12:0-20:0,...,0,0,0,0,0,1,0,0,0,0
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,11:0-2:0,...,0,0,0,0,0,0,0,0,0,0
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,10:30-21:0,...,1,0,0,0,0,1,0,0,0,0
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,9:0-9:15,...,0,0,0,0,0,0,3,3,0,0
4,GgWFjRHhelaiUgR2-H6N4g,pRPT3vqhqpU7kHgmKJamvw,-_GnwXmzC3DXsHR9nyaC2g,3,0,0,0,3.5 stars! I got the avocado margarita and it ...,2012-11-06 07:09:57,11:0-21:0,...,0,0,0,0,0,1,0,0,0,0
5,ldGCo22ax_ZvHV8TgE27Pg,Df1kUPqooB1FrnwvXi5HsQ,Qydynzc1ApxsSkVNEgPs9A,1,0,1,0,I love ale house. I'm here all the time. The 1...,2014-07-13 18:59:44,11:0-2:0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
dfin['date'] = pd.to_datetime(dfin.date)

In [8]:
from IPython.display import display

pd.options.display.max_columns = None
display(dfin)

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,hours.Wednesday,attributes.RestaurantsDelivery,attributes.Open24Hours,attributes.DogsAllowed,attributes.CoatCheck,postal_code,attributes.Smoking,hours.Thursday,attributes.DietaryRestrictions,city,hours.Tuesday,attributes.RestaurantsGoodForGroups,attributes.BYOB,attributes.AgesAllowed,hours.Friday,stars_y,latitude,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.HairSpecializesIn,attributes.RestaurantsCounterService,attributes.Corkage,attributes.GoodForKids,categories,name_x,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,is_open,attributes.Ambience,longitude,hours.Monday,attributes.BestNights,hours.Saturday,review_count_x,attributes.ByAppointmentOnly,attributes.NoiseLevel,state,attributes.DriveThru,attributes.HasTV,attributes.WiFi,attributes.Music,hours,attributes.BusinessParking,address,attributes.RestaurantsAttire,hours.Sunday,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.AcceptsInsurance,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,12:0-20:0,True,NaN,False,NaN,30318,NaN,12:0-20:0,NaN,Atlanta,12:0-20:0,True,NaN,NaN,12:0-21:0,4.0,33.798343,u'full_bar',2.0,True,NaN,NaN,NaN,True,"Burgers, Specialty Food, Restaurants, Barbeque...",FLIP burger boutique,NaN,True,True,True,NaN,1,"{'touristy': False, 'hipster': False, 'romanti...",-84.415749,12:0-20:0,NaN,12:0-21:0,1909,False,u'average',GA,False,True,u'no',NaN,"{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...","{'garage': False, 'street': False, 'validated'...",1587 Howell Mill Rd,u'casual',12:0-21:0,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,NaN,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,None,Britney,30,2011-08-18 17:30:19,33,18,8,NaN,"wc42I8BztdQZp1-tl_fUWg, TCbUlrORknmlKO06OZjTFw...",0,3.26,0,0,0,0,0,0,1,0,0,0,0
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,11:0-2:0,True,NaN,False,False,30309,u'outdoor',11:0-2:0,NaN,Atlanta,11:0-2:0,True,NaN,NaN,11:0-2:0,3.0,33.780594,u'full_bar',2.0,True,NaN,NaN,NaN,True,"Venues & Event Spaces, Restaurants, Sports Bar...",Hudson Grille,NaN,True,True,True,NaN,1,"{'touristy': False, 'hipster': False, 'romanti...",-84.384349,11:0-2:0,"{'monday': False, 'tuesday': False, 'friday': ...",11:0-2:0,505,NaN,u'loud',GA,NaN,True,u'free',"{'dj': False, 'background_music': False, 'no_m...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...","{'garage': None, 'street': None, 'validated': ...",942 Peachtree St NE,u'casual',11:0-0:0,"{'dessert': None, 'latenight': True, 'lunch': ...",False,False,NaN,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,True,Jordan,2,2015-06-23 16:31:11,0,0,0,NaN,"0boxx-XaP3yCLdZLCWepQA, emm2NazzH6gtJbnUejjsGg...",0,4.50,0,0,0,0,0,0,0,0,0,0,0
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,10:30-21:0,True,NaN,NaN,NaN,78758,NaN,10:30-21:0,NaN,Austin,10:30-21:0,True,NaN,NaN,10:30-21:0,4.0,30.361116,u'none',1.0,True,NaN,NaN,False,True,"Vietnamese, Restaurants",Pho Van,NaN,NaN,NaN,True,'yes_free',1,"{'romantic': False, 'intimate': False, 'touris...",-97.715703,10:30-21:0,NaN,10:30-21:0,364,NaN,u'average',TX,NaN,True,u'no',NaN,

In [9]:
#Also it is good to drop unneccessary columns

dfin = dfin.drop(['hours.Monday','postal_code','hours.Thursday','hours.Tuesday','hours.Friday','hours.Wednesday',
             'hours.Saturday','hours.Sunday','longitude','latitude'], axis=1)


In [10]:
display(dfin)

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,attributes.RestaurantsDelivery,attributes.Open24Hours,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,attributes.DietaryRestrictions,city,attributes.RestaurantsGoodForGroups,attributes.BYOB,attributes.AgesAllowed,stars_y,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.HairSpecializesIn,attributes.RestaurantsCounterService,attributes.Corkage,attributes.GoodForKids,categories,name_x,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,is_open,attributes.Ambience,attributes.BestNights,review_count_x,attributes.ByAppointmentOnly,attributes.NoiseLevel,state,attributes.DriveThru,attributes.HasTV,attributes.WiFi,attributes.Music,hours,attributes.BusinessParking,address,attributes.RestaurantsAttire,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.AcceptsInsurance,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,True,NaN,False,NaN,NaN,NaN,Atlanta,True,NaN,NaN,4.0,u'full_bar',2.0,True,NaN,NaN,NaN,True,"Burgers, Specialty Food, Restaurants, Barbeque...",FLIP burger boutique,NaN,True,True,True,NaN,1,"{'touristy': False, 'hipster': False, 'romanti...",NaN,1909,False,u'average',GA,False,True,u'no',NaN,"{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...","{'garage': False, 'street': False, 'validated'...",1587 Howell Mill Rd,u'casual',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,NaN,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,None,Britney,30,2011-08-18 17:30:19,33,18,8,NaN,"wc42I8BztdQZp1-tl_fUWg, TCbUlrORknmlKO06OZjTFw...",0,3.26,0,0,0,0,0,0,1,0,0,0,0
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,True,NaN,False,False,u'outdoor',NaN,Atlanta,True,NaN,NaN,3.0,u'full_bar',2.0,True,NaN,NaN,NaN,True,"Venues & Event Spaces, Restaurants, Sports Bar...",Hudson Grille,NaN,True,True,True,NaN,1,"{'touristy': False, 'hipster': False, 'romanti...","{'monday': False, 'tuesday': False, 'friday': ...",505,NaN,u'loud',GA,NaN,True,u'free',"{'dj': False, 'background_music': False, 'no_m...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...","{'garage': None, 'street': None, 'validated': ...",942 Peachtree St NE,u'casual',"{'dessert': None, 'latenight': True, 'lunch': ...",False,False,NaN,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,True,Jordan,2,2015-06-23 16:31:11,0,0,0,NaN,"0boxx-XaP3yCLdZLCWepQA, emm2NazzH6gtJbnUejjsGg...",0,4.50,0,0,0,0,0,0,0,0,0,0,0
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,True,NaN,NaN,NaN,NaN,NaN,Austin,True,NaN,NaN,4.0,u'none',1.0,True,NaN,NaN,False,True,"Vietnamese, Restaurants",Pho Van,NaN,NaN,NaN,True,'yes_free',1,"{'romantic': False, 'intimate': False, 'touris...",NaN,364,NaN,u'average',TX,NaN,True,u'no',NaN,"{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:...","{'garage': False, 'street': False, 'validated'...",8557 Research Blvd,'casual',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,False,NaN,False,"{'BusinessParking': ""{'garage': False, 'street...",True,True,False,David,144,2011-06-19 17:31:54,64,6,18,NaN,"q4atAOkgDhm3i5BosJ6cDw, 6kWWcge84BkEhorBH8qWcQ...",2,3.70,0,1,0,0,0,0,1,0,0,0,0
3,WxrK52MRJy6C

In [11]:
#Here we can see which columns contain null values and how many there are

for col in dfin.columns:
    print(col, dfin[col].isnull().sum())

review_id 0
user_id 0
business_id 0
stars_x 0
useful_x 0
funny_x 0
cool_x 0
text 0
date 0
attributes.RestaurantsDelivery 17186
attributes.Open24Hours 212298
attributes.DogsAllowed 65604
attributes.CoatCheck 160858
attributes.Smoking 170270
attributes.DietaryRestrictions 214874
city 0
attributes.RestaurantsGoodForGroups 21979
attributes.BYOB 173528
attributes.AgesAllowed 214114
stars_y 0
attributes.Alcohol 18166
attributes.RestaurantsPriceRange2 7784
attributes.RestaurantsTableService 79475
attributes.HairSpecializesIn 214874
attributes.RestaurantsCounterService 212298
attributes.Corkage 154723
attributes.GoodForKids 12794
categories 0
name_x 0
attributes.BusinessAcceptsBitcoin 144102
attributes.HappyHour 56814
attributes.WheelchairAccessible 104413
attributes.BusinessAcceptsCreditCards 2926
attributes.BYOBCorkage 156434
is_open 0
attributes.Ambience 23060
attributes.BestNights 151474
review_count_x 0
attributes.ByAppointmentOnly 121790
attributes.NoiseLevel 20117
state 0
attributes.Dri

In [12]:
#Delete those with very null values (limit 180k)

dfin = dfin.drop(['attributes.AcceptsInsurance', 'attributes.DietaryRestrictions', 'attributes.Open24Hours', 'attributes.AgesAllowed', 'attributes.HairSpecializesIn','attributes.RestaurantsCounterService'], axis = 1)

In [13]:
dfin

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,attributes.RestaurantsDelivery,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,city,attributes.RestaurantsGoodForGroups,attributes.BYOB,stars_y,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.Corkage,attributes.GoodForKids,categories,name_x,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,is_open,attributes.Ambience,attributes.BestNights,review_count_x,attributes.ByAppointmentOnly,attributes.NoiseLevel,state,attributes.DriveThru,attributes.HasTV,attributes.WiFi,attributes.Music,hours,attributes.BusinessParking,address,attributes.RestaurantsAttire,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,True,False,NaN,NaN,Atlanta,True,NaN,4.0,u'full_bar',2.0,True,NaN,True,"Burgers, Specialty Food, Restaurants, Barbeque...",FLIP burger boutique,NaN,True,True,True,NaN,1,"{'touristy': False, 'hipster': False, 'romanti...",NaN,1909,False,u'average',GA,False,True,u'no',NaN,"{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...","{'garage': False, 'street': False, 'validated'...",1587 Howell Mill Rd,u'casual',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,None,Britney,30,2011-08-18 17:30:19,33,18,8,NaN,"wc42I8BztdQZp1-tl_fUWg, TCbUlrORknmlKO06OZjTFw...",0,3.26,0,0,0,0,0,0,1,0,0,0,0
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,True,False,False,u'outdoor',Atlanta,True,NaN,3.0,u'full_bar',2.0,True,NaN,True,"Venues & Event Spaces, Restaurants, Sports Bar...",Hudson Grille,NaN,True,True,True,NaN,1,"{'touristy': False, 'hipster': False, 'romanti...","{'monday': False, 'tuesday': False, 'friday': ...",505,NaN,u'loud',GA,NaN,True,u'free',"{'dj': False, 'background_music': False, 'no_m...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...","{'garage': None, 'street': None, 'validated': ...",942 Peachtree St NE,u'casual',"{'dessert': None, 'latenight': True, 'lunch': ...",False,False,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,True,Jordan,2,2015-06-23 16:31:11,0,0,0,NaN,"0boxx-XaP3yCLdZLCWepQA, emm2NazzH6gtJbnUejjsGg...",0,4.50,0,0,0,0,0,0,0,0,0,0,0
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,True,NaN,NaN,NaN,Austin,True,NaN,4.0,u'none',1.0,True,False,True,"Vietnamese, Restaurants",Pho Van,NaN,NaN,NaN,True,'yes_free',1,"{'romantic': False, 'intimate': False, 'touris...",NaN,364,NaN,u'average',TX,NaN,True,u'no',NaN,"{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:...","{'garage': False, 'street': False, 'validated'...",8557 Research Blvd,'casual',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,False,False,"{'BusinessParking': ""{'garage': False, 'street...",True,True,False,David,144,2011-06-19 17:31:54,64,6,18,NaN,"q4atAOkgDhm3i5BosJ6cDw, 6kWWcge84BkEhorBH8qWcQ...",2,3.70,0,1,0,0,0,0,1,0,0,0,0
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,NaN,False,NaN,NaN,Orlando,NaN,NaN,2.0,NaN,2.0,NaN,NaN,True,"Resorts, Hotels, Active Life, Event Planning &

In [14]:
#Find only those columns that are like attributes - (several answer options)

atributy = [col for col in dfin if col.startswith('attribute')]
atributy

['attributes.RestaurantsDelivery',
 'attributes.DogsAllowed',
 'attributes.CoatCheck',
 'attributes.Smoking',
 'attributes.RestaurantsGoodForGroups',
 'attributes.BYOB',
 'attributes.Alcohol',
 'attributes.RestaurantsPriceRange2',
 'attributes.RestaurantsTableService',
 'attributes.Corkage',
 'attributes.GoodForKids',
 'attributes.BusinessAcceptsBitcoin',
 'attributes.HappyHour',
 'attributes.WheelchairAccessible',
 'attributes.BusinessAcceptsCreditCards',
 'attributes.BYOBCorkage',
 'attributes.Ambience',
 'attributes.BestNights',
 'attributes.ByAppointmentOnly',
 'attributes.NoiseLevel',
 'attributes.DriveThru',
 'attributes.HasTV',
 'attributes.WiFi',
 'attributes.Music',
 'attributes.BusinessParking',
 'attributes.RestaurantsAttire',
 'attributes.GoodForMeal',
 'attributes.GoodForDancing',
 'attributes.Caters',
 'attributes.RestaurantsReservations',
 'attributes',
 'attributes.RestaurantsTakeOut',
 'attributes.BikeParking',
 'attributes.OutdoorSeating']

In [15]:
dfatr = dfin[atributy]
dfatr

,attributes.RestaurantsDelivery,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,attributes.RestaurantsGoodForGroups,attributes.BYOB,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.Corkage,attributes.GoodForKids,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,attributes.Ambience,attributes.BestNights,attributes.ByAppointmentOnly,attributes.NoiseLevel,attributes.DriveThru,attributes.HasTV,attributes.WiFi,attributes.Music,attributes.BusinessParking,attributes.RestaurantsAttire,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,True,False,NaN,NaN,True,NaN,u'full_bar',2.0,True,NaN,True,NaN,True,True,True,NaN,"{'touristy': False, 'hipster': False, 'romanti...",NaN,False,u'average',False,True,u'no',NaN,"{'garage': False, 'street': False, 'validated'...",u'casual',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,None
1,True,False,False,u'outdoor',True,NaN,u'full_bar',2.0,True,NaN,True,NaN,True,True,True,NaN,"{'touristy': False, 'hipster': False, 'romanti...","{'monday': False, 'tuesday': False, 'friday': ...",NaN,u'loud',NaN,True,u'free',"{'dj': False, 'background_music': False, 'no_m...","{'garage': None, 'street': None, 'validated': ...",u'casual',"{'dessert': None, 'latenight': True, 'lunch': ...",False,False,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,True
2,True,NaN,NaN,NaN,True,NaN,u'none',1.0,True,False,True,NaN,NaN,NaN,True,'yes_free',"{'romantic': False, 'intimate': False, 'touris...",NaN,NaN,u'average',NaN,True,u'no',NaN,"{'garage': False, 'street': False, 'validated'...",'casual',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,False,False,"{'BusinessParking': ""{'garage': False, 'street...",True,True,False
3,NaN,False,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,True,NaN,NaN,NaN,True,NaN,NaN,NaN,True,NaN,NaN,NaN,'free',NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,NaN,"{'DogsAllowed': 'False', 'BusinessParking': ""{...",NaN,True,NaN
4,True,True,NaN,NaN,True,NaN,'full_bar',2.0,True,False,True,NaN,True,True,True,'no',"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,u'average',False,True,'free',NaN,"{'garage': False, 'street': True, 'validated':...",'casual',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,False,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214869,False,True,False,u'no',True,False,'full_bar',2.0,True,False,True,False,True,True,True,'no',"{'touristy': False, 'hipster': None, 'romantic...","{'monday': False, 'tuesday': False, 'friday': ...",False,u'average',NaN,True,u'free',"{'dj': None, 'background_music': False, 'no_mu...","{'garage': False, 'street': True, 'validated':...",'casual',"{'dessert': None, 'latenight': False, 'lunch':...",False,False,False,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,True
214870,False,True,False,u'no',True,False,'full_bar',2.0,True,False,True,False,True,True,True,'no',"{'touristy': False, 'hipster': None, 'romantic...","{'monday': False, 'tuesday': False, 'friday': ...",False,u'average',NaN,True,u'free',"{'dj': None, 'background_music': False, 'no_mu...","{'garage': False, 'street': True, 'validated':...",'casual',"{'dessert': None, 'latenight': False, 'lunch':...",False,False,False,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,True
214871,None,False,NaN,NaN,False,NaN,'beer_and_wine',2.0,NaN,True,False,NaN,False,NaN,False,'yes_corkage',"{'touristy': False, 'hipster': False, 'romanti...",NaN,False,u'average',NaN,False,u'no',NaN,"{'gara

In [16]:
#Missing values change to False (if it is not mentioned there, we do not count on it being there)

for column in dfatr:
    dfatr[column].fillna(False, inplace=True)
dfatr

C:\Users\Dell\anaconda\lib\site-packages\pandas\core\series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,attributes.RestaurantsDelivery,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,attributes.RestaurantsGoodForGroups,attributes.BYOB,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.Corkage,attributes.GoodForKids,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,attributes.Ambience,attributes.BestNights,attributes.ByAppointmentOnly,attributes.NoiseLevel,attributes.DriveThru,attributes.HasTV,attributes.WiFi,attributes.Music,attributes.BusinessParking,attributes.RestaurantsAttire,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,True,False,False,False,True,False,u'full_bar',2,True,False,True,False,True,True,True,False,"{'touristy': False, 'hipster': False, 'romanti...",False,False,u'average',False,True,u'no',False,"{'garage': False, 'street': False, 'validated'...",u'casual',"{'dessert': False, 'latenight': False, 'lunch'...",False,True,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,None
1,True,False,False,u'outdoor',True,False,u'full_bar',2,True,False,True,False,True,True,True,False,"{'touristy': False, 'hipster': False, 'romanti...","{'monday': False, 'tuesday': False, 'friday': ...",False,u'loud',False,True,u'free',"{'dj': False, 'background_music': False, 'no_m...","{'garage': None, 'street': None, 'validated': ...",u'casual',"{'dessert': None, 'latenight': True, 'lunch': ...",False,False,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,True
2,True,False,False,False,True,False,u'none',1,True,False,True,False,False,False,True,'yes_free',"{'romantic': False, 'intimate': False, 'touris...",False,False,u'average',False,True,u'no',False,"{'garage': False, 'street': False, 'validated'...",'casual',"{'dessert': False, 'latenight': False, 'lunch'...",False,False,False,"{'BusinessParking': ""{'garage': False, 'street...",True,True,False
3,False,False,False,False,False,False,False,2,False,False,True,False,False,False,True,False,False,False,True,False,False,False,'free',False,"{'garage': False, 'street': False, 'validated'...",False,False,False,False,False,"{'DogsAllowed': 'False', 'BusinessParking': ""{...",False,True,False
4,True,True,False,False,True,False,'full_bar',2,True,False,True,False,True,True,True,'no',"{'touristy': False, 'hipster': False, 'romanti...",False,False,u'average',False,True,'free',False,"{'garage': False, 'street': True, 'validated':...",'casual',"{'dessert': False, 'latenight': False, 'lunch'...",False,True,False,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214869,False,True,False,u'no',True,False,'full_bar',2,True,False,True,False,True,True,True,'no',"{'touristy': False, 'hipster': None, 'romantic...","{'monday': False, 'tuesday': False, 'friday': ...",False,u'average',False,True,u'free',"{'dj': None, 'background_music': False, 'no_mu...","{'garage': False, 'street': True, 'validated':...",'casual',"{'dessert': None, 'latenight': False, 'lunch':...",False,False,False,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,True
214870,False,True,False,u'no',True,False,'full_bar',2,True,False,True,False,True,True,True,'no',"{'touristy': False, 'hipster': None, 'romantic...","{'monday': False, 'tuesday': False, 'friday': ...",False,u'average',False,True,u'free',"{'dj': None, 'background_music': False, 'no_mu...","{'garage': False, 'street': True, 'validated':...",'casual',"{'dessert': None, 'latenight': False, 'lunch':...",False,False,False,"{'Alcohol': ""'full_bar'"", 'HasTV': 'True', 'No...",True,True,True
214871,None,False,False,False,False,False,'beer_and_wine',2,False,True,False,False,False,False,

In [17]:
#Intermediate step for boolean

for column in dfin[atributy]:
    dfin[column].replace('no', False)

dfin


,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,attributes.RestaurantsDelivery,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,city,attributes.RestaurantsGoodForGroups,attributes.BYOB,stars_y,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.Corkage,attributes.GoodForKids,categories,name_x,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,is_open,attributes.Ambience,attributes.BestNights,review_count_x,attributes.ByAppointmentOnly,attributes.NoiseLevel,state,attributes.DriveThru,attributes.HasTV,attributes.WiFi,attributes.Music,hours,attributes.BusinessParking,address,attributes.RestaurantsAttire,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,True,False,NaN,NaN,Atlanta,True,NaN,4.0,u'full_bar',2.0,True,NaN,True,"Burgers, Specialty Food, Restaurants, Barbeque...",FLIP burger boutique,NaN,True,True,True,NaN,1,"{'touristy': False, 'hipster': False, 'romanti...",NaN,1909,False,u'average',GA,False,True,u'no',NaN,"{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...","{'garage': False, 'street': False, 'validated'...",1587 Howell Mill Rd,u'casual',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,None,Britney,30,2011-08-18 17:30:19,33,18,8,NaN,"wc42I8BztdQZp1-tl_fUWg, TCbUlrORknmlKO06OZjTFw...",0,3.26,0,0,0,0,0,0,1,0,0,0,0
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,True,False,False,u'outdoor',Atlanta,True,NaN,3.0,u'full_bar',2.0,True,NaN,True,"Venues & Event Spaces, Restaurants, Sports Bar...",Hudson Grille,NaN,True,True,True,NaN,1,"{'touristy': False, 'hipster': False, 'romanti...","{'monday': False, 'tuesday': False, 'friday': ...",505,NaN,u'loud',GA,NaN,True,u'free',"{'dj': False, 'background_music': False, 'no_m...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...","{'garage': None, 'street': None, 'validated': ...",942 Peachtree St NE,u'casual',"{'dessert': None, 'latenight': True, 'lunch': ...",False,False,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,True,Jordan,2,2015-06-23 16:31:11,0,0,0,NaN,"0boxx-XaP3yCLdZLCWepQA, emm2NazzH6gtJbnUejjsGg...",0,4.50,0,0,0,0,0,0,0,0,0,0,0
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,True,NaN,NaN,NaN,Austin,True,NaN,4.0,u'none',1.0,True,False,True,"Vietnamese, Restaurants",Pho Van,NaN,NaN,NaN,True,'yes_free',1,"{'romantic': False, 'intimate': False, 'touris...",NaN,364,NaN,u'average',TX,NaN,True,u'no',NaN,"{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:...","{'garage': False, 'street': False, 'validated'...",8557 Research Blvd,'casual',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,False,False,"{'BusinessParking': ""{'garage': False, 'street...",True,True,False,David,144,2011-06-19 17:31:54,64,6,18,NaN,"q4atAOkgDhm3i5BosJ6cDw, 6kWWcge84BkEhorBH8qWcQ...",2,3.70,0,1,0,0,0,0,1,0,0,0,0
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,NaN,False,NaN,NaN,Orlando,NaN,NaN,2.0,NaN,2.0,NaN,NaN,True,"Resorts, Hotels, Active Life, Event Planning &

In [18]:
#What attributes have what values and how many there are 

for column in dfin[atributy]:
    print(dfin[column].value_counts())

True     123766
False     42139
None      31783
Name: attributes.RestaurantsDelivery, dtype: int64
False    116279
True      32475
None        516
Name: attributes.DogsAllowed, dtype: int64
False    46746
True      7270
Name: attributes.CoatCheck, dtype: int64
u'no'         33938
u'outdoor'    10055
u'yes'          611
Name: attributes.Smoking, dtype: int64
True     173120
False     19775
Name: attributes.RestaurantsGoodForGroups, dtype: int64
False    33507
True      7839
Name: attributes.BYOB, dtype: int64
u'full_bar'         74227
'full_bar'          51571
u'beer_and_wine'    37516
'none'              13108
u'none'             10846
'beer_and_wine'      9440
Name: attributes.Alcohol, dtype: int64
2.0    150913
1.0     31190
3.0     21382
4.0      3605
Name: attributes.RestaurantsPriceRange2, dtype: int64
True     107441
False     27958
Name: attributes.RestaurantsTableService, dtype: int64
False    38481
True     21670
Name: attributes.Corkage, dtype: int64
True     145801
False    

'casual'     99717
u'casual'    82690
'dressy'      4597
u'dressy'     4161
Name: attributes.RestaurantsAttire, dtype: int64
{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}     31648
{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': True, 'brunch': False, 'breakfast': False}    15898
{'dessert': False, 'latenight': False, 'lunch': None, 'dinner': True, 'brunch': False, 'breakfast': False}     10304
{'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}       9305
{'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}        9249
                                                                                                               ...  
{'dessert': False, 'latenight': True, 'lunch': True, 'dinner': None, 'brunch': False, 'breakfast': False}        327
{'dessert': False, 'latenight': None, 'lunch': None, 'di

Name: attributes.OutdoorSeating, dtype: int64


In [19]:
#Correct editing

#RestaurantsDelivery
dfin['attributes.RestaurantsDelivery'] = dfin['attributes.RestaurantsDelivery'].replace("None", 'False')
dfin['attributes.RestaurantsDelivery'] = dfin['attributes.RestaurantsDelivery'].astype(str)

#DogsAllowed
dfin['attributes.DogsAllowed'] = dfin['attributes.DogsAllowed'].replace("None", 'False')
dfin['attributes.DogsAllowed'] = dfin['attributes.DogsAllowed'].astype(str)

#coat check
dfin['attributes.CoatCheck'] = dfin['attributes.CoatCheck'].astype(bool)

#Smoking
dfin['attributes.Smoking'] = dfin['attributes.Smoking'].replace("u'no'", 'False')
dfin['attributes.Smoking'] = dfin['attributes.Smoking'].replace("u'yes'", 'True')
dfin['attributes.Smoking'] = dfin['attributes.Smoking'].replace("u'outdoor'", 'outdoor')
dfin['attributes.Smoking'] = dfin['attributes.Smoking'].astype(str)

#RestaurantsGoodForGroups
dfin['attributes.RestaurantsGoodForGroups'] = dfin['attributes.RestaurantsGoodForGroups'].astype(bool)

#BYOB (?)
dfin['attributes.BYOB'] = dfin['attributes.BYOB'].astype(bool)

#attributes.Alcohol
dfin['attributes.Alcohol'] = dfin['attributes.Alcohol'].replace("u'full_bar'", 'full_bar')
dfin['attributes.Alcohol'] = dfin['attributes.Alcohol'].replace("'full_bar'", 'full_bar')
dfin['attributes.Alcohol'] = dfin['attributes.Alcohol'].replace("u'beer_and_wine'", 'beer_and_wine')
dfin['attributes.Alcohol'] = dfin['attributes.Alcohol'].replace("'beer_and_wine'", 'beer_and_wine')
dfin['attributes.Alcohol'] = dfin['attributes.Alcohol'].replace("u'none'", 'False')
dfin['attributes.Alcohol'] = dfin['attributes.Alcohol'].replace("'none'", 'none')
dfin['attributes.Alcohol'] = dfin['attributes.Alcohol'].replace("none", 'False')
dfin['attributes.Alcohol'] = dfin['attributes.Alcohol'].astype(str)

#attributes.RestaurantsTableService
dfin['attributes.RestaurantsTableService'] = dfin['attributes.RestaurantsTableService'].astype(bool)

#attributes.Corkage
dfin['attributes.Corkage'] = dfin['attributes.Corkage'].astype(bool)

#attributes.GoodForKids
dfin['attributes.GoodForKids'] = dfin['attributes.GoodForKids'].astype(bool)

#attributes.BusinessAcceptsBitcoin
dfin['attributes.BusinessAcceptsBitcoin'] = dfin['attributes.BusinessAcceptsBitcoin'].astype(bool)

#attributes.HappyHour
dfin['attributes.HappyHour'] = dfin['attributes.HappyHour'].astype(bool)

#attributes.WheelchairAccessible
dfin['attributes.WheelchairAccessible'] = dfin['attributes.WheelchairAccessible'].astype(bool)

#attributes.BusinessAcceptsCreditCards
dfin['attributes.BusinessAcceptsCreditCards'] = dfin['attributes.BusinessAcceptsCreditCards'].astype(bool)

#attributes.BYOBCorkage
dfin['attributes.BYOBCorkage'] = dfin['attributes.BYOBCorkage'].replace("'no'",'False')
dfin['attributes.BYOBCorkage'] = dfin['attributes.BYOBCorkage'].replace("'yes_corkage'", 'yes_charged')
dfin['attributes.BYOBCorkage'] = dfin['attributes.BYOBCorkage'].replace("'yes_free'", 'yes_free')
dfin['attributes.BYOBCorkage'] = dfin['attributes.BYOBCorkage'].astype(str)


#ByAppointmentOnly
dfin['attributes.ByAppointmentOnly'] = dfin['attributes.ByAppointmentOnly'].astype(bool)

#attributes.NoiseLevel
dfin['attributes.NoiseLevel'] = dfin['attributes.NoiseLevel'].replace("u'average'",'average')
dfin['attributes.NoiseLevel'] = dfin['attributes.NoiseLevel'].replace("'average'",'average')
dfin['attributes.NoiseLevel'] = dfin['attributes.NoiseLevel'].replace("u'loud'",'loud')
dfin['attributes.NoiseLevel'] = dfin['attributes.NoiseLevel'].replace("u'very_loud'",'very_loud')
dfin['attributes.NoiseLevel'] = dfin['attributes.NoiseLevel'].replace("u'quiet'",'quiet')
dfin['attributes.NoiseLevel'] = dfin['attributes.NoiseLevel'].replace("'quiet'",'quiet')
dfin['attributes.NoiseLevel'] = dfin['attributes.NoiseLevel'].replace("'loud'",'loud')
dfin['attributes.NoiseLevel'] = dfin['attributes.NoiseLevel'].astype(str)

#attributes.DriveThru
dfin['attributes.DriveThru'] = dfin['attributes.DriveThru'].astype(bool)

#attributes.HasTV
dfin['attributes.HasTV'] = dfin['attributes.HasTV'].astype(bool)

#attributes.WiFi
dfin['attributes.WiFi'] = dfin['attributes.WiFi'].replace("u'no'", 'False')
dfin['attributes.WiFi'] = dfin['attributes.WiFi'].replace("'no'", 'False')
dfin['attributes.WiFi'] = dfin['attributes.WiFi'].replace("None", 'False')
dfin['attributes.WiFi'] = dfin['attributes.WiFi'].replace("u'free'", 'free')
dfin['attributes.WiFi'] = dfin['attributes.WiFi'].replace("'free'", 'free')
dfin['attributes.WiFi'] = dfin['attributes.WiFi'].replace("u'paid'", 'charged')
dfin['attributes.WiFi'] = dfin['attributes.WiFi'].astype(str)

#attributes.RestaurantsAttire
dfin['attributes.RestaurantsAttire'] = dfin['attributes.RestaurantsAttire'].replace("u'dressy'", 'dressy')
dfin['attributes.RestaurantsAttire'] = dfin['attributes.RestaurantsAttire'].replace("u'casual'", 'casual')
dfin['attributes.RestaurantsAttire'] = dfin['attributes.RestaurantsAttire'].replace("'dressy'", 'dressy')
dfin['attributes.RestaurantsAttire'] = dfin['attributes.RestaurantsAttire'].replace("'casual'", 'casual')
dfin['attributes.RestaurantsAttire'] = dfin['attributes.RestaurantsAttire'].replace('nan', 'False')
dfin['attributes.RestaurantsAttire'] = dfin['attributes.RestaurantsAttire'].astype(str)

#attributes.GoodForDancing
dfin['attributes.GoodForDancing'] = dfin['attributes.GoodForDancing'].astype(bool)

#attributes.Caters
dfin['attributes.Caters'] = dfin['attributes.Caters'].astype(bool)

#attributes.RestaurantsReservations
dfin['attributes.RestaurantsReservations'] = dfin['attributes.RestaurantsReservations'].replace("None", 'False')
dfin['attributes.RestaurantsReservations'] = dfin['attributes.RestaurantsReservations'].astype(str)

#attributes.RestaurantsTakeOut
dfin['attributes.RestaurantsTakeOut'] = dfin['attributes.RestaurantsTakeOut'].replace("None",'False')
dfin['attributes.RestaurantsTakeOut'] = dfin['attributes.RestaurantsTakeOut'].astype(str)

#attributes.BikeParking
dfin['attributes.BikeParking'] = dfin['attributes.BikeParking'].astype(bool)

#attributes.OutdoorSeating
dfin['attributes.OutdoorSeating'] = dfin['attributes.OutdoorSeating'].replace("None", 'False')
dfin['attributes.OutdoorSeating'] = dfin['attributes.OutdoorSeating'].astype(str)

In [20]:
for column in dfin[atributy]:
    print(dfin[column].value_counts())

True     123766
False     73922
nan       17186
Name: attributes.RestaurantsDelivery, dtype: int64
False    116795
nan       65604
True      32475
Name: attributes.DogsAllowed, dtype: int64
True     168128
False     46746
Name: attributes.CoatCheck, dtype: int64
nan        170270
False       33938
outdoor     10055
True          611
Name: attributes.Smoking, dtype: int64
True     195099
False     19775
Name: attributes.RestaurantsGoodForGroups, dtype: int64
True     181367
False     33507
Name: attributes.BYOB, dtype: int64
full_bar         125798
beer_and_wine     46956
False             23954
nan               18166
Name: attributes.Alcohol, dtype: int64
2.0    150913
1.0     31190
3.0     21382
4.0      3605
Name: attributes.RestaurantsPriceRange2, dtype: int64
True     186916
False     27958
Name: attributes.RestaurantsTableService, dtype: int64
True     176393
False     38481
Name: attributes.Corkage, dtype: int64
True     158595
False     56279
Name: attributes.GoodForKids, dtype

Name: attributes.GoodForDancing, dtype: int64
True     122143
False     92731
Name: attributes.Caters, dtype: int64
True     102800
False     89450
nan       22624
Name: attributes.RestaurantsReservations, dtype: int64
{'Alcohol': "u'full_bar'", 'HasTV': 'True', 'NoiseLevel': "u'loud'", 'RestaurantsAttire': "u'casual'", 'BusinessAcceptsCreditCards': 'True', 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}", 'RestaurantsGoodForGroups': 'True', 'Caters': 'False', 'WiFi': "u'free'", 'RestaurantsReservations': 'False', 'BYOB': 'True', 'BusinessAcceptsBitcoin': 'False', 'BikeParking': 'True', 'Corkage': 'False', 'RestaurantsTakeOut': 'True', 'GoodForKids': 'True', 'HappyHour': 'True', 'DriveThru': 'False', 'DogsAllowed': 'False', 'RestaurantsTableService': 'True', 'OutdoorSeating': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsDelivery': 'True', 'ByAppointmentOnly'

In [21]:
dftest = dfin


In [22]:
dfin = dftest.fillna(False)
dfin = dftest.replace('nan', 'False')

In [23]:
##What attributes have what values and how many there are after correction
for column in dfin[atributy]:
    print(dfin[column].value_counts())

True     123766
False     91108
Name: attributes.RestaurantsDelivery, dtype: int64
False    182399
True      32475
Name: attributes.DogsAllowed, dtype: int64
True     168128
False     46746
Name: attributes.CoatCheck, dtype: int64
False      204208
outdoor     10055
True          611
Name: attributes.Smoking, dtype: int64
True     195099
False     19775
Name: attributes.RestaurantsGoodForGroups, dtype: int64
True     181367
False     33507
Name: attributes.BYOB, dtype: int64
full_bar         125798
beer_and_wine     46956
False             42120
Name: attributes.Alcohol, dtype: int64
2.0    150913
1.0     31190
3.0     21382
4.0      3605
Name: attributes.RestaurantsPriceRange2, dtype: int64
True     186916
False     27958
Name: attributes.RestaurantsTableService, dtype: int64
True     176393
False     38481
Name: attributes.Corkage, dtype: int64
True     158595
False     56279
Name: attributes.GoodForKids, dtype: int64
True     144849
False     70025
Name: attributes.BusinessAcceptsBi

{'Alcohol': "u'full_bar'", 'HasTV': 'True', 'NoiseLevel': "u'loud'", 'RestaurantsAttire': "u'casual'", 'BusinessAcceptsCreditCards': 'True', 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}", 'RestaurantsGoodForGroups': 'True', 'Caters': 'False', 'WiFi': "u'free'", 'RestaurantsReservations': 'False', 'BYOB': 'True', 'BusinessAcceptsBitcoin': 'False', 'BikeParking': 'True', 'Corkage': 'False', 'RestaurantsTakeOut': 'True', 'GoodForKids': 'True', 'HappyHour': 'True', 'DriveThru': 'False', 'DogsAllowed': 'False', 'RestaurantsTableService': 'True', 'OutdoorSeating': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsDelivery': 'True', 'ByAppointmentOnly': 'False', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'BusinessParking': "{'garage': True, 'street': True, 'validated': True, 'lot': Fals

In [24]:
#Distribution of ambient attributes

import json

dfin['attributes.Ambience'] = dfin['attributes.Ambience'].replace(np.NaN,
        "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}")

dfin['attributes.Ambience'] = dfin['attributes.Ambience'].astype(str)
dfin['attributes.Ambience'] = dfin['attributes.Ambience'].map(lambda x: dict(eval(x)))
dfAmb = pd.json_normalize(dfin['attributes.Ambience'])
dfAmb = dfAmb.replace(np.nan, False) 
dfAmb = dfAmb.add_prefix('Ambience_')
dfAmb.head(5)


,Ambience_touristy,Ambience_hipster,Ambience_romantic,Ambience_divey,Ambience_intimate,Ambience_trendy,Ambience_upscale,Ambience_classy,Ambience_casual
0,False,False,False,False,False,True,False,True,False
1,False,False,False,False,False,False,False,True,True
2,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,True


In [25]:
#Distribution of Goodformeal attributes

dfin['attributes.GoodForMeal'] = dfin['attributes.GoodForMeal'].replace(np.NaN,
        "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}")

dfin['attributes.GoodForMeal'] = dfin['attributes.GoodForMeal'].astype(str)
dfin['attributes.GoodForMeal'] = dfin['attributes.GoodForMeal'].map(lambda x: dict(eval(x)))
dGFM = pd.json_normalize(dfin['attributes.GoodForMeal'])
dGFM = dGFM.fillna(value=np.nan)
dGFM = dGFM.replace(np.nan, False) 
dGFM = dGFM.add_prefix('Good_Meals_')
dGFM.head(5)

,Good_Meals_dessert,Good_Meals_latenight,Good_Meals_lunch,Good_Meals_dinner,Good_Meals_brunch,Good_Meals_breakfast
0,False,False,True,True,False,False
1,False,True,True,True,False,False
2,False,False,True,True,False,False
3,False,False,False,False,False,False
4,False,False,True,True,False,False


In [26]:
display(dfin)

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,attributes.RestaurantsDelivery,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,city,attributes.RestaurantsGoodForGroups,attributes.BYOB,stars_y,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.Corkage,attributes.GoodForKids,categories,name_x,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,is_open,attributes.Ambience,attributes.BestNights,review_count_x,attributes.ByAppointmentOnly,attributes.NoiseLevel,state,attributes.DriveThru,attributes.HasTV,attributes.WiFi,attributes.Music,hours,attributes.BusinessParking,address,attributes.RestaurantsAttire,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.RestaurantsReservations,attributes,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,True,False,True,False,Atlanta,True,True,4.0,full_bar,2.0,True,True,True,"Burgers, Specialty Food, Restaurants, Barbeque...",FLIP burger boutique,True,True,True,True,False,1,"{'touristy': False, 'hipster': False, 'romanti...",NaN,1909,False,average,GA,False,True,False,NaN,"{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...","{'garage': False, 'street': False, 'validated'...",1587 Howell Mill Rd,casual,"{'dessert': False, 'latenight': False, 'lunch'...",True,True,True,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,False,Britney,30,2011-08-18 17:30:19,33,18,8,NaN,"wc42I8BztdQZp1-tl_fUWg, TCbUlrORknmlKO06OZjTFw...",0,3.26,0,0,0,0,0,0,1,0,0,0,0
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,True,False,False,outdoor,Atlanta,True,True,3.0,full_bar,2.0,True,True,True,"Venues & Event Spaces, Restaurants, Sports Bar...",Hudson Grille,True,True,True,True,False,1,"{'touristy': False, 'hipster': False, 'romanti...","{'monday': False, 'tuesday': False, 'friday': ...",505,True,loud,GA,True,True,free,"{'dj': False, 'background_music': False, 'no_m...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...","{'garage': None, 'street': None, 'validated': ...",942 Peachtree St NE,casual,"{'dessert': None, 'latenight': True, 'lunch': ...",False,False,False,"{'Alcohol': ""u'full_bar'"", 'HasTV': 'True', 'N...",True,True,True,Jordan,2,2015-06-23 16:31:11,0,0,0,NaN,"0boxx-XaP3yCLdZLCWepQA, emm2NazzH6gtJbnUejjsGg...",0,4.50,0,0,0,0,0,0,0,0,0,0,0
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,True,False,True,False,Austin,True,True,4.0,False,1.0,True,False,True,"Vietnamese, Restaurants",Pho Van,True,True,True,True,yes_free,1,"{'romantic': False, 'intimate': False, 'touris...",NaN,364,True,average,TX,True,True,False,NaN,"{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:...","{'garage': False, 'street': False, 'validated'...",8557 Research Blvd,casual,"{'dessert': False, 'latenight': False, 'lunch'...",True,False,False,"{'BusinessParking': ""{'garage': False, 'street...",True,True,False,David,144,2011-06-19 17:31:54,64,6,18,NaN,"q4atAOkgDhm3i5BosJ6cDw, 6kWWcge84BkEhorBH8qWcQ...",2,3.70,0,1,0,0,0,0,1,0,0,0,0
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,False,False,True,False,Orlando,True,True,2.0,False,2.0,True,True,True,"Resorts, Hotels, Active Life, Event Pla

In [27]:
#Distribution of Businessparking attributes
dfin['attributes.BusinessParking'] = dfin['attributes.BusinessParking'].replace(np.NaN,
        "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}")

dfin['attributes.BusinessParking'] = dfin['attributes.BusinessParking'].astype(str)
dfin['attributes.BusinessParking'] = dfin['attributes.BusinessParking'].map(lambda x: dict(eval(x)))
dBP = pd.json_normalize(dfin['attributes.BusinessParking'])
dBP = dBP.fillna(value=np.nan)
dBP = dBP.replace(np.nan, False) 
dBP = dBP.add_prefix('BP_')
dBP.head(5)

,BP_garage,BP_street,BP_validated,BP_lot,BP_valet
0,False,False,False,True,False
1,False,False,False,False,True
2,False,False,False,True,False
3,False,False,False,True,False
4,False,True,False,True,False


In [28]:
#Delete another unneccessary columns and also 'attributes' - they are contained in another columns

dfin = dfin.drop(['attributes.BestNights', 'attributes.DriveThru', 'attributes'], axis = 1)
display(dfin)

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,attributes.RestaurantsDelivery,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,city,attributes.RestaurantsGoodForGroups,attributes.BYOB,stars_y,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.Corkage,attributes.GoodForKids,categories,name_x,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,is_open,attributes.Ambience,review_count_x,attributes.ByAppointmentOnly,attributes.NoiseLevel,state,attributes.HasTV,attributes.WiFi,attributes.Music,hours,attributes.BusinessParking,address,attributes.RestaurantsAttire,attributes.GoodForMeal,attributes.GoodForDancing,attributes.Caters,attributes.RestaurantsReservations,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,True,False,True,False,Atlanta,True,True,4.0,full_bar,2.0,True,True,True,"Burgers, Specialty Food, Restaurants, Barbeque...",FLIP burger boutique,True,True,True,True,False,1,"{'touristy': False, 'hipster': False, 'romanti...",1909,False,average,GA,True,False,NaN,"{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...","{'garage': False, 'street': False, 'validated'...",1587 Howell Mill Rd,casual,"{'dessert': False, 'latenight': False, 'lunch'...",True,True,True,True,True,False,Britney,30,2011-08-18 17:30:19,33,18,8,NaN,"wc42I8BztdQZp1-tl_fUWg, TCbUlrORknmlKO06OZjTFw...",0,3.26,0,0,0,0,0,0,1,0,0,0,0
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,True,False,False,outdoor,Atlanta,True,True,3.0,full_bar,2.0,True,True,True,"Venues & Event Spaces, Restaurants, Sports Bar...",Hudson Grille,True,True,True,True,False,1,"{'touristy': False, 'hipster': False, 'romanti...",505,True,loud,GA,True,free,"{'dj': False, 'background_music': False, 'no_m...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...","{'garage': None, 'street': None, 'validated': ...",942 Peachtree St NE,casual,"{'dessert': None, 'latenight': True, 'lunch': ...",False,False,False,True,True,True,Jordan,2,2015-06-23 16:31:11,0,0,0,NaN,"0boxx-XaP3yCLdZLCWepQA, emm2NazzH6gtJbnUejjsGg...",0,4.50,0,0,0,0,0,0,0,0,0,0,0
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,True,False,True,False,Austin,True,True,4.0,False,1.0,True,False,True,"Vietnamese, Restaurants",Pho Van,True,True,True,True,yes_free,1,"{'romantic': False, 'intimate': False, 'touris...",364,True,average,TX,True,False,NaN,"{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:...","{'garage': False, 'street': False, 'validated'...",8557 Research Blvd,casual,"{'dessert': False, 'latenight': False, 'lunch'...",True,False,False,True,True,False,David,144,2011-06-19 17:31:54,64,6,18,NaN,"q4atAOkgDhm3i5BosJ6cDw, 6kWWcge84BkEhorBH8qWcQ...",2,3.70,0,1,0,0,0,0,1,0,0,0,0
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,False,False,True,False,Orlando,True,True,2.0,False,2.0,True,True,True,"Resorts, Hotels, Active Life, Event Planning &...",CoCo Key Hotel and Water Resort,True,True,True,True,False,1,"{'touristy': False, 'hipster': False, 'romanti...",310,True,False,FL,True,free,NaN,"{'Monday': '9:0-9:15', 'Tuesday': '9:0-9:15', ...","{'garage': False, 'street': False, 'validated'...",7400 International Dr,False,"{'

In [29]:
dfin = pd.concat([dfin,dGFM,dfAmb,dBP],axis=1)
dfin = dfin.drop(['attributes.GoodForMeal', 'attributes.Ambience','attributes.BusinessParking'], axis=1)
dfin

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,attributes.RestaurantsDelivery,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,city,attributes.RestaurantsGoodForGroups,attributes.BYOB,stars_y,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.Corkage,attributes.GoodForKids,categories,name_x,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,is_open,review_count_x,attributes.ByAppointmentOnly,attributes.NoiseLevel,state,attributes.HasTV,attributes.WiFi,attributes.Music,hours,address,attributes.RestaurantsAttire,attributes.GoodForDancing,attributes.Caters,attributes.RestaurantsReservations,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,Good_Meals_dessert,Good_Meals_latenight,Good_Meals_lunch,Good_Meals_dinner,Good_Meals_brunch,Good_Meals_breakfast,Ambience_touristy,Ambience_hipster,Ambience_romantic,Ambience_divey,Ambience_intimate,Ambience_trendy,Ambience_upscale,Ambience_classy,Ambience_casual,BP_garage,BP_street,BP_validated,BP_lot,BP_valet
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,True,False,True,False,Atlanta,True,True,4.0,full_bar,2.0,True,True,True,"Burgers, Specialty Food, Restaurants, Barbeque...",FLIP burger boutique,True,True,True,True,False,1,1909,False,average,GA,True,False,NaN,"{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...",1587 Howell Mill Rd,casual,True,True,True,True,True,False,Britney,30,2011-08-18 17:30:19,33,18,8,NaN,"wc42I8BztdQZp1-tl_fUWg, TCbUlrORknmlKO06OZjTFw...",0,3.26,0,0,0,0,0,0,1,0,0,0,0,False,False,True,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,True,False,False,outdoor,Atlanta,True,True,3.0,full_bar,2.0,True,True,True,"Venues & Event Spaces, Restaurants, Sports Bar...",Hudson Grille,True,True,True,True,False,1,505,True,loud,GA,True,free,"{'dj': False, 'background_music': False, 'no_m...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...",942 Peachtree St NE,casual,False,False,False,True,True,True,Jordan,2,2015-06-23 16:31:11,0,0,0,NaN,"0boxx-XaP3yCLdZLCWepQA, emm2NazzH6gtJbnUejjsGg...",0,4.50,0,0,0,0,0,0,0,0,0,0,0,False,True,True,True,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,True
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,True,False,True,False,Austin,True,True,4.0,False,1.0,True,False,True,"Vietnamese, Restaurants",Pho Van,True,True,True,True,yes_free,1,364,True,average,TX,True,False,NaN,"{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:...",8557 Research Blvd,casual,True,False,False,True,True,False,David,144,2011-06-19 17:31:54,64,6,18,NaN,"q4atAOkgDhm3i5BosJ6cDw, 6kWWcge84BkEhorBH8qWcQ...",2,3.70,0,1,0,0,0,0,1,0,0,0,0,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,False,False,True,False,Orlando,True,True,2.0,False,2.0,True,True,True,"Resorts, Hotels, Active Life, Event Planning &...",CoCo Key Hotel and Water Resort,True,True,True,True,False,1,310,True,False,FL,True,free,NaN,"{'Monday': '9:0-9:15', 'Tuesday': '9:0-9:15', ...",7400 Internati

In [30]:
dfin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214874 entries, 0 to 214873
Data columns (total 87 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   review_id                              214874 non-null  object        
 1   user_id                                214874 non-null  object        
 2   business_id                            214874 non-null  object        
 3   stars_x                                214874 non-null  int64         
 4   useful_x                               214874 non-null  int64         
 5   funny_x                                214874 non-null  int64         
 6   cool_x                                 214874 non-null  int64         
 7   text                                   214874 non-null  object        
 8   date                                   214874 non-null  datetime64[ns]
 9   attributes.RestaurantsDelivery         214874 no

In [31]:
#Store the new dataset with which we are going to work

dfin.to_csv('dfin.csv')

In [32]:
dfin.head()

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,attributes.RestaurantsDelivery,attributes.DogsAllowed,attributes.CoatCheck,attributes.Smoking,city,attributes.RestaurantsGoodForGroups,attributes.BYOB,stars_y,attributes.Alcohol,attributes.RestaurantsPriceRange2,attributes.RestaurantsTableService,attributes.Corkage,attributes.GoodForKids,categories,name_x,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.BusinessAcceptsCreditCards,attributes.BYOBCorkage,is_open,review_count_x,attributes.ByAppointmentOnly,attributes.NoiseLevel,state,attributes.HasTV,attributes.WiFi,attributes.Music,hours,address,attributes.RestaurantsAttire,attributes.GoodForDancing,attributes.Caters,attributes.RestaurantsReservations,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,Good_Meals_dessert,Good_Meals_latenight,Good_Meals_lunch,Good_Meals_dinner,Good_Meals_brunch,Good_Meals_breakfast,Ambience_touristy,Ambience_hipster,Ambience_romantic,Ambience_divey,Ambience_intimate,Ambience_trendy,Ambience_upscale,Ambience_classy,Ambience_casual,BP_garage,BP_street,BP_validated,BP_lot,BP_valet
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,True,False,True,False,Atlanta,True,True,4.0,full_bar,2.0,True,True,True,"Burgers, Specialty Food, Restaurants, Barbeque...",FLIP burger boutique,True,True,True,True,False,1,1909,False,average,GA,True,False,NaN,"{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...",1587 Howell Mill Rd,casual,True,True,True,True,True,False,Britney,30,2011-08-18 17:30:19,33,18,8,NaN,"wc42I8BztdQZp1-tl_fUWg, TCbUlrORknmlKO06OZjTFw...",0,3.26,0,0,0,0,0,0,1,0,0,0,0,False,False,True,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,True,False,False,outdoor,Atlanta,True,True,3.0,full_bar,2.0,True,True,True,"Venues & Event Spaces, Restaurants, Sports Bar...",Hudson Grille,True,True,True,True,False,1,505,True,loud,GA,True,free,"{'dj': False, 'background_music': False, 'no_m...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...",942 Peachtree St NE,casual,False,False,False,True,True,True,Jordan,2,2015-06-23 16:31:11,0,0,0,NaN,"0boxx-XaP3yCLdZLCWepQA, emm2NazzH6gtJbnUejjsGg...",0,4.50,0,0,0,0,0,0,0,0,0,0,0,False,True,True,True,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,True
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,True,False,True,False,Austin,True,True,4.0,False,1.0,True,False,True,"Vietnamese, Restaurants",Pho Van,True,True,True,True,yes_free,1,364,True,average,TX,True,False,NaN,"{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:...",8557 Research Blvd,casual,True,False,False,True,True,False,David,144,2011-06-19 17:31:54,64,6,18,NaN,"q4atAOkgDhm3i5BosJ6cDw, 6kWWcge84BkEhorBH8qWcQ...",2,3.70,0,1,0,0,0,0,1,0,0,0,0,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,False,False,True,False,Orlando,True,True,2.0,False,2.0,True,True,True,"Resorts, Hotels, Active Life, Event Planning &...",CoCo Key Hotel and Water Resort,True,True,True,True,False,1,310,True,False,FL,True,free,NaN,"{'Monday': '9:0-9:15', 'Tuesday': '9:0-9:15', ...",7400 Internati